In [1]:
import os
import ee
import json
import geemap

ee.Authenticate()
ee.Initialize(project=os.getenv('GEE_PROJECT'))

In [2]:
# Load Panama border from shapefile and GeoJSON. We will only work within the Panama border.
with open('./assets/geo/pa.geometry.geojson') as f:
    pa_geojson = json.load(f)

pa_geometry = ee.Geometry(pa_geojson)

In [3]:
# This loads the satellite embeddings dataset for Panama
embeddings = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL").filterBounds(pa_geometry)

def get_year_embedding(year: int) -> ee.Image:
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')
    return (embeddings
            .filter(ee.Filter.date(start, end))
            .filterBounds(pa_geometry)
            .mosaic())

def export_thumb(img: ee.Image, fname: str, vis: dict, dimensions: int = 4096):
    url = img.clip(pa_geometry).getThumbUrl({
        'region': pa_geometry,
        'dimensions': dimensions,
        'format': 'png',
        **vis,
    })
    print(fname, url)

In [4]:
emb24 = get_year_embedding(2024)
emb23 = get_year_embedding(2023)

In [ ]:
culebra_cut = ee.Geometry.Point([-79.68097726517239, 9.089013216557404])

with open('./assets/geo/water.samples.geojson') as f:
    clean_waters_geojson = json.load(f)
water_regions_geo = ee.Geometry(clean_waters_geojson)

# Embeddings for points of interest. They look like objects mapping bands to values.
culebra_sample = emb24.sample(region=culebra_cut, scale=10).first().toDictionary()
water_samples = emb24.sample(
    region=water_regions_geo,
    scale=10,
    numPixels=100,
    seed=12345
)

In [12]:
vis_rgb = {
    "bands": ["A01", "A16", "A09"],  # or your ["A21","A10","A09"]
    "min": -0.3,
    "max": 0.3,
}

emb17 = get_year_embedding(2017)
emb24 = get_year_embedding(2024)

export_thumb(emb17, "panama_2017_rgb", vis_rgb)
export_thumb(emb24, "panama_2024_rgb", vis_rgb)

panama_2017_rgb https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/0360a73cd39f5a3ad487174e8b09f1e6-315b63860f75e693ebe80f020e86982e:getPixels
panama_2024_rgb https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/da452ef3033d77cdc6fed68e116278f5-d46d3c0f0d5456d776614603b7c18099:getPixels


In [25]:
# training = emb24.sample({
#     'region': pa_geometry,
#     'scale': 10,
#     'numPixels': 5000,  # tweak if needed
#     'seed': 0,
# })

training = emb24.sample(
    region=pa_geometry,
    scale=20,
    numPixels=2500,
    seed=0
)

# Train KMeans in 64-D embedding space
n_clusters = 6  # usually 5–8 works nicely
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

clusters = emb24.cluster(clusterer)
clusters_vis = clusters.randomVisualizer()
cluster_img_viz = {
    "bands": ["viz-red", "viz-green", "viz-blue"],
    "min": -0.3,
    "max": 0.3,
}

export_thumb(clusters_vis, "panama_2024_clusters", cluster_img_viz, dimensions=2048)

panama_2024_clusters https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/002aac738ac72d24deb2d3fc1ab3d3c2-4d0bdb77f3d529340d83cf927ab66b60:getPixels


In [5]:
band_names = emb24.bandNames()
# Embedding vector for Culebra point, but as an ee.Array
culebra_water = ee.Array(culebra_sample.values(band_names))

mean_dict = water_samples.reduceColumns(
    reducer=ee.Reducer.mean().repeat(band_names.size()),
    selectors=band_names
).get('mean')
mu_water = ee.Array(mean_dict) # length-64 vector of mean water embeddings

In [6]:
dot = ee.Number(culebra_water.multiply(mu_water).reduce(ee.Reducer.sum(), [0]).getInfo()[0])
mu_norm_sq = ee.Number(mu_water.multiply(mu_water).reduce(ee.Reducer.sum(), [0]).getInfo()[0])
projection = mu_water.multiply(dot.divide(mu_norm_sq))
culebra_prime = culebra_water.subtract(projection)

In [7]:
def cosine_similarity(image, vector):
    # Convert the image to an array image.
    image_array = image.toArray()
    # Multiply the array image by the vector. This is element-wise.
    dot = image_array.multiply(vector).arrayReduce(ee.Reducer.sum(), [0])
    
    image_norm = image_array.pow(2).arrayReduce(ee.Reducer.sum(), [0]).sqrt()
    vector_norm = vector.pow(2).reduce(ee.Reducer.sum(), [0]).sqrt()
    
    # The result of the dot product and norms are single-band images.
    return dot.divide(image_norm.multiply(vector_norm)).arrayGet(0)

similarity_map = cosine_similarity(emb24, culebra_water)

In [29]:
export_thumb(similarity_map.clip(pa_geometry), "panama_2024_similarity_culebra", {'min': 0, 'max': 1, 'palette': ['000004', '2C105C', '711F81', 'B63679','EE605E', 'FDAE78', 'FCFDBF', 'FFFFFF']})

panama_2024_similarity_culebra https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/e3a6ed1a499d20ed85cb2e9fc3bf0144-047a48ef85ad34ca8f674ad202aac9a8:getPixels


In [9]:
vis = {
    "bands": ["A21", "A10", "A09"],
    "min": -0.3,
    "max": 0.3,
}

import leafmap.maplibregl as leafmap
Map = leafmap.Map(projection="globe", style="3d-terrain")

Map.add_ee_layer(emb24.clip(pa_geometry), vis, "2024")
Map.set_terrain(exaggeration=5)

Map

Html(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass': True, 'showZoom': True, 'visual…

In [10]:
Map.to_html(output="./assets/panama_2024_embedding.html")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 967172: character maps to <undefined>

In [34]:
from IPython.display import Image
url = similarity_map.getThumbUrl({
    'min': 0,
    'max': 1,
    'palette': ['000004', '2C105C', '711F81', 'B63679','EE605E', 'FDAE78', 'FCFDBF', 'FFFFFF'],
    'region': pa_geometry,
    'dimensions': 4096,
})
print(url)


# print('wait')
# Image(url=url)

https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/4383aa89a066bebb6d95927252915957-164e71f6015cc968caa22e511105d1a2:getPixels
